In [33]:
import import_ipynb

In [34]:
%run NLTK.ipynb import Tokenize,LoweringStemming,BagOfWord  # Use %run to import other jupyte notebook file

In [35]:
AllWords = []
Tags = []
XY = []

XTrain = []
YTrain = []

In [36]:
import json
with open("intents.json", "r") as F:
    intents = json.load(F)
intents

{'intents': [{'tag': 'greeting',
   'patterns': ['Hi',
    'Hey',
    'How are you',
    'Is anyone there?',
    'Hello',
    'Good day'],
   'responses': ['Hey :-)',
    'Hello, thanks for visiting',
    'Hi there, what can I do for you?',
    'Hi there, how can I help?']},
  {'tag': 'goodbye',
   'patterns': ['Bye', 'See you later', 'Goodbye'],
   'responses': ['See you later, thanks for visiting',
    'Have a nice day',
    'Bye! Come back again soon.']},
  {'tag': 'thanks',
   'patterns': ['Thanks', 'Thank you', "That's helpful", "Thank's a lot!"],
   'responses': ['Happy to help!', 'Any time!', 'My pleasure']},
  {'tag': 'items',
   'patterns': ['Which items do you have?',
    'What kinds of items are there?',
    'What do you sell?'],
   'responses': ['We sell coffee and tea', 'We have coffee and tea']},
  {'tag': 'payments',
   'patterns': ['Do you take credit cards?',
    'Do you accept Mastercard?',
    'Can I pay with Paypal?',
    'Are you cash only?'],
   'responses': ['We 

In [37]:
for intent in intents["intents"]:
    Tag = intent["tag"]
    Tags.append(Tag)
    for Pattern in intent["patterns"]:
        Words = Tokenize(Pattern)
        Words = [LoweringStemming(Stem) for Stem in Words if(Stem not in ["?","!",".",",","'s"])]
        AllWords.extend(Words)  #Words is a list
        XY.append((Words,Tag))
        print(Words)

AllWords = sorted(set(AllWords)) #Set() for remove duplicate
print("AllWords : ",AllWords)
print("XY : ",XY)

['hi']
['hey']
['how', 'are', 'you']
['is', 'anyon', 'there']
['hello']
['good', 'day']
['bye']
['see', 'you', 'later']
['goodby']
['thank']
['thank', 'you']
['that', 'help']
['thank', 'a', 'lot']
['which', 'item', 'do', 'you', 'have']
['what', 'kind', 'of', 'item', 'are', 'there']
['what', 'do', 'you', 'sell']
['do', 'you', 'take', 'credit', 'card']
['do', 'you', 'accept', 'mastercard']
['can', 'i', 'pay', 'with', 'paypal']
['are', 'you', 'cash', 'onli']
['how', 'long', 'doe', 'deliveri', 'take']
['how', 'long', 'doe', 'ship', 'take']
['when', 'do', 'i', 'get', 'my', 'deliveri']
['tell', 'me', 'a', 'joke']
['tell', 'me', 'someth', 'funni']
['do', 'you', 'know', 'a', 'joke']
AllWords :  ['a', 'accept', 'anyon', 'are', 'bye', 'can', 'card', 'cash', 'credit', 'day', 'deliveri', 'do', 'doe', 'funni', 'get', 'good', 'goodby', 'have', 'hello', 'help', 'hey', 'hi', 'how', 'i', 'is', 'item', 'joke', 'kind', 'know', 'later', 'long', 'lot', 'mastercard', 'me', 'my', 'of', 'onli', 'pay', 'paypal

In [38]:
import numpy as np
for (Pattern,Tag) in XY:
    XTrain.append(BagOfWord(Pattern,AllWords))
    YTrain.append(Tags.index(Tag))  
XTrain = np.array(XTrain)
YTrain = np.array(YTrain)

In [39]:
XTrain

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 1.]], dtype=float32)

In [40]:
YTrain

array([0, 0, 0, 0, 0, 0, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 4, 4, 4, 4, 5, 5,
       5, 6, 6, 6])

In [41]:
from torch.utils.data import Dataset,DataLoader
class ChatDataset(Dataset):
    # def __init__(self):
    #     self.NSample = len(XTrain)
    #     self.XData = XTrain
    #     self.YData = YTrain

    # def __getitem__(self,index):
    #     return self.XData[index],self.YData[index]

    def __init__(self):
        self.NSample = len(XTrain)
        self.XData = XTrain.astype(np.float32)  # Ensure the correct dtype
        self.YData = YTrain

    def __getitem__(self, index):
        x = torch.from_numpy(self.XData[index])
        y = torch.tensor(self.YData[index], dtype=torch.long)  # Ensure correct dtype

    def __len__(self):
        return self.NSample

In [70]:
BatchSize = 8
DataSet = ChatDataset()
if __name__ == '__main__':
    
TrainLoader = DataLoader(dataset=DataSet, batch_size=BatchSize, shuffle=True)
print(TrainLoader)

NameError: name 'freeze_support' is not defined

In [64]:
%run Model.ipynb import NeuralNet

In [65]:
InputSize = len(AllWords)
HiddenSize = 8 
NumClasses = len(Tags)
print(InputSize, HiddenSize, NumClasses)

Device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
Model = NeuralNet(InputSize, HiddenSize, NumClasses).to(Device)
print(Device)

5 8 7
cuda


In [68]:

LearningRate = 0.001
NumEpochs = 1000

Criterian = nn.CrossEntropyLoss()
Optimizer = torch.optim.Adam(Model.parameters(), lr=LearningRate)

for Epochs in range(NumEpochs):
    for (Words,Labels) in TrainLoader:
        Words = Words.to(Device)
        Labels = Labels.to(device).long()   

        Output = Model(Words)
        Loss = Criterian(Output, Labels)

        Optimizer.zero_grad()
        Loss.backward()
        Optimizer.step()

    if(Epochs + 1) % 100 == 0 :
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


x_data type: <class 'numpy.ndarray'>
x_data[index] type: <class 'numpy.ndarray'>
x_data[index] shape: (54,)


RuntimeError: Numpy is not available